# Derivation of working with Euler Parameters
T. Fitzgerald

In [1]:
using Pkg
Pkg.activate(".")
using SymPy
using LinearAlgebra

  Activating project at `c:\Users\tim\Documents\2025FallExamples\11-14`


## Define the rotations and build $C$

In [2]:
β0, β1, β2, β3 = symbols("beta_0 beta_1 beta_2 beta_3", real=true)
r1,r2,r3 = symbols("r_1 r_2 r_3", real=true)

(r_1, r_2, r_3)

In [3]:
Dot(a,b) = sum(a .* b)
struct quaternion
    r
    v
end
function Base.:+(a::quaternion, b::quaternion)
    return quaternion( a.r + b.r, a.v + b.v)
end
function Base.:*(a::quaternion, b::quaternion)
    return quaternion(a.r*b.r - Dot(a.v, b.v), a.r*b.v + b.r*a.v + cross(a.v, b.v))
end

In [4]:
q = quaternion(β0, [β1,β2,β3])
qinv = quaternion(β0, -[β1, β2, β3])

quaternion(beta_0, Sym{PyCall.PyObject}[-beta_1, -beta_2, -beta_3])

In [5]:
p = quaternion(0,[r1,r2,r3])
pp = q*p*qinv

quaternion(beta_0*(-beta_1*r_1 - beta_2*r_2 - beta_3*r_3) + beta_1*(beta_0*r_1 + beta_2*r_3 - beta_3*r_2) + beta_2*(beta_0*r_2 - beta_1*r_3 + beta_3*r_1) + beta_3*(beta_0*r_3 + beta_1*r_2 - beta_2*r_1), Sym{PyCall.PyObject}[beta_0*(beta_0*r_1 + beta_2*r_3 - beta_3*r_2) - beta_1*(-beta_1*r_1 - beta_2*r_2 - beta_3*r_3) + beta_2*(beta_0*r_3 + beta_1*r_2 - beta_2*r_1) - beta_3*(beta_0*r_2 - beta_1*r_3 + beta_3*r_1), beta_0*(beta_0*r_2 - beta_1*r_3 + beta_3*r_1) - beta_1*(beta_0*r_3 + beta_1*r_2 - beta_2*r_1) - beta_2*(-beta_1*r_1 - beta_2*r_2 - beta_3*r_3) + beta_3*(beta_0*r_1 + beta_2*r_3 - beta_3*r_2), beta_0*(beta_0*r_3 + beta_1*r_2 - beta_2*r_1) + beta_1*(beta_0*r_2 - beta_1*r_3 + beta_3*r_1) - beta_2*(beta_0*r_1 + beta_2*r_3 - beta_3*r_2) - beta_3*(-beta_1*r_1 - beta_2*r_2 - beta_3*r_3)])

In [6]:
pp.r |> simplify

0

In [7]:
pp.v |> simplify

3-element Vector{Sym{PyCall.PyObject}}:
 beta_0*(beta_0*r_1 + beta_2*r_3 - beta_3*r_2) - beta_1*(-beta_1*r_1 - beta_2*r_2 - beta_3*r_3) + beta_2*(beta_0*r_3 + beta_1*r_2 - beta_2*r_1) - beta_3*(beta_0*r_2 - beta_1*r_3 + beta_3*r_1)
 beta_0*(beta_0*r_2 - beta_1*r_3 + beta_3*r_1) - beta_1*(beta_0*r_3 + beta_1*r_2 - beta_2*r_1) - beta_2*(-beta_1*r_1 - beta_2*r_2 - beta_3*r_3) + beta_3*(beta_0*r_1 + beta_2*r_3 - beta_3*r_2)
 beta_0*(beta_0*r_3 + beta_1*r_2 - beta_2*r_1) + beta_1*(beta_0*r_2 - beta_1*r_3 + beta_3*r_1) - beta_2*(beta_0*r_1 + beta_2*r_3 - beta_3*r_2) - beta_3*(-beta_1*r_1 - beta_2*r_2 - beta_3*r_3)

In [8]:
C = [diff(v, r) |> simplify for r in [r1, r2, r3], v in pp.v]

3×3 Matrix{Sym{PyCall.PyObject}}:
 beta_0^2 + beta_1^2 - beta_2^2 - beta_3^2  …         -2*beta_0*beta_2 + 2*beta_1*beta_3
        -2*beta_0*beta_3 + 2*beta_1*beta_2             2*beta_0*beta_1 + 2*beta_2*beta_3
         2*beta_0*beta_2 + 2*beta_1*beta_3     beta_0^2 - beta_1^2 - beta_2^2 + beta_3^2

In [9]:
# from class we got:
tilde(x) = [ 0 -x[3] x[2]; x[3] 0 -x[1]; -x[2] x[1] 0 ]
C_alt = q.v*transpose(q.v) + β0^2*I - 2*β0*tilde(q.v) + tilde(q.v)^2

3×3 Matrix{Sym{PyCall.PyObject}}:
 beta_0^2 + beta_1^2 - beta_2^2 - beta_3^2  …         -2*beta_0*beta_2 + 2*beta_1*beta_3
        -2*beta_0*beta_3 + 2*beta_1*beta_2             2*beta_0*beta_1 + 2*beta_2*beta_3
         2*beta_0*beta_2 + 2*beta_1*beta_3     beta_0^2 - beta_1^2 - beta_2^2 + beta_3^2

In [10]:
C - C_alt

3×3 Matrix{Sym{PyCall.PyObject}}:
 0  0  0
 0  0  0
 0  0  0

In [11]:
det(C) |> subs(β0^2 => 1 - β1^2 - β2^2 - β3^2) |> simplify

1

In [31]:
[ t |> subs(β0^2 => 1 - β1^2 - β2^2 - β3^2) |> simplify
 for t in transpose(C) - inv(C) ]

3×3 Matrix{Sym{PyCall.PyObject}}:
 0  0  0
 0  0  0
 0  0  0